<a href="https://colab.research.google.com/github/balling/ml-hackpack/blob/master/Chest_X_Ray.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

First, change your runtime to GPU so that the network is trained faster!

TODO: add screenshots

Make sure you run the two cells below!

In [1]:
#@title Import the necessary libraries
import os
import numpy as np
import pandas as pd
from skimage.io import imread 
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras.datasets import mnist
from keras.utils import plot_model
from numpy.random import seed
from tensorflow import set_random_seed

Using TensorFlow backend.


In [2]:
#@title Download the images
!git clone https://github.com/balling/ml-hackpack.git

Cloning into 'ml-hackpack'...
remote: Enumerating objects: 245, done.
remote: Total 245 (delta 0), reused 0 (delta 0), pack-reused 245
Receiving objects: 100% (245/245), 75.92 MiB | 18.06 MiB/s, done.
Resolving deltas: 100% (15/15), done.


In [0]:

seed(42)
set_random_seed(42)

NUM_IMG = 25

def get_training_data(train_path, labels_path):
	train_images = []
	train_files = []
	for filename in os.listdir(train_path):
		if filename.endswith(".png"):
			train_files.append(train_path + filename)

	features = []
		
	for i, train_file in enumerate(train_files):
			if i >= NUM_IMG: break
			train_image = imread(train_file, True)
			feature_set = np.asarray(train_image)
			features.append(feature_set)

	labels_df = pd.read_csv(labels_path) #["Finding Labels"]
	labels_df = labels_df["Finding Labels"]
	labels = np.zeros(NUM_IMG) # 0 for no finding, 1 for finding.

	# loading all labels
	for i in range(NUM_IMG):
		if (labels_df[i] == 'No Finding'):
			labels[i] = 0
		else:
			labels[i] = 1
	images = np.expand_dims(np.array(features), axis=3).astype('float32') / 255 # adding single channel
	return images, labels
	
X_train, y_train = get_training_data("ml-hackpack/data/train/", "ml-hackpack/data/train-labels.csv")
X_test, y_test = get_training_data("ml-hackpack/data/test/", "ml-hackpack/data/test-labels.csv")

model = Sequential()

model.add(Conv2D(4, (3, 3), strides=(2,2), activation='relu', input_shape=(1024, 1024, 1), data_format='channels_last'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(4, (3, 3), strides=(2,2), activation='relu'))
# model.add(Dropout(0.25))
 
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
 
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
 
model.fit(X_train, y_train, batch_size=4, nb_epoch=10, verbose=1)
 
score = model.evaluate(X_test, y_test, verbose=0)


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:51: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Epoch 1/10
25/25 [==============================] - 13s 501ms/step - loss: 0.6958 - acc: 0.4400
Epoch 2/10
25/25 [==============================] - 9s 359ms/step - loss: 0.6063 - acc: 1.0000
Epoch 3/10
25/25 [==============================] - 9s 355ms/step - loss: 0.3974 - acc: 0.9200
Epoch 4/10
25/25 [==============================] - 9s 360ms/step - loss: 0.2265 - acc: 0.8800
Epoch 5/10
25/25 [==============================] - 9s 359ms/step - loss: 0.0896 - acc: 1.0000
Epoch 6/10
25/25 [==============================] - 9s 353ms/step - loss: 0.0151 - acc: 1.0000
Epoch 7/10
25/25 [==============================] - 9s 356ms/step - loss: 0.0025 - acc: 1.0000
Epoch 8/10
24/25 [===========================>..] - ETA: 0s - loss: 8.0660e-04 - acc: 1.0000